# **Exporting NetCDF Timesteps as GeoTIFFs**

## **Introduction**
In this notebook, we will extract **daily chlorophyll-a data** from a Zarr file and export each **time slice as an individual GeoTIFF**. This allows us to style each GeoTiff separately and automate the map export within a QGIS map layout. You may decide to now download only a smaller window of data from the E.U. Copernicus Marine Service website.

By the end of this tutorial, you will:
- Know how to **open and explore Zarr files** using `xarray`.
- Export **each timestep** of a variable as a **separate GeoTIFF**.

## 📑 Table of Contents

[🧰 1. Import Required Libraries](#1-import-required-libraries)  
[📦 2. Define File Paths and Load Data](#2-define-file-paths-and-load-data)  
[🔍 3. Inspect and Prepare the NetCDF File](#3-inspect-and-prepare-the-netcdf-file)  
[🗂️ 4. Loop Through Timesteps and Export Rasters](#4-loop-through-timesteps-and-export-rasters)  


## 🧰 **1. Import Required Libraries**

For this project, we use libraries from three key categories:  

- **Data analysis libraries**  
  These help us efficiently process and analyze structured and geospatial data:  
  - `pandas` – Handles **tabular data** like CSV files and spreadsheets.  
  - `xarray` – Works with **labeled N-dimensional arrays**, making it ideal for **Zarr** and multidimensional datasets.  
  - `rioxarray` – Extends `xarray` with **geospatial capabilities**, allowing us to read, write, and manipulate raster datasets.  

- **Path and pattern handling**  
  To work efficiently with files and filenames, we use:  
  - `pathlib.Path` – A modern, object-oriented way to **handle file paths**.  
  - `re` – Provides **regular expressions** for pattern matching and parsing filenames.  

- **Logging and system utilities**  
  Instead of using `print()`, we’ll use a library called `loguru` to:  
  - Show clear status messages as each step runs.  
  - Display color-coded messages like 🎉 success or 🚩 errors, which help with debugging and tracking progress.  
  - Keep our output clean, especially in notebooks.

We also configure the logger to remove any background colors, which can look distracting in Jupyter, and to show simple, readable messages for each step.

In [ ]:
# --- Data analysis libraries ---
import pandas as pd  # Tabular data manipulation and analysis
import xarray as xr  # Labeled N-dimensional arrays
import rioxarray as rxr  # Geo-aware extension for xarray, handles raster I/O

# --- Path and pattern handling ---
from pathlib import Path  # Object-oriented file paths
import re  # Regular expressions for pattern matching and string parsing

# --- Logging and system utilities ---
from loguru import logger  # Simple, colorful logging
import sys  # For routing logger output

# Set up the logger to show clean messages (colored text, no background)
from helpers import configure_logger
configure_logger()

# Check that everything was imported successfully
logger.success("🎉 Libraries successfully imported.")

## 📦 **2. Define File Paths**

In [ ]:
# File paths
zarr_path = Path.cwd() / "data/" / "cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m_chl_180.00W-179.75E_80.00S-90.00N_0.49m_2024-01-01-2024-12-31.zarr"
output_dir = Path.cwd() / "data" / "rasters"

# Ensure the output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Load Zarr
ds = xr.open_zarr(zarr_path)

## 🔍 **3. Inspect and Prepare the Zarr File**

In [ ]:
# Extract base name before the date range (e.g., _2024-03-01-2024-06-01)
match = re.match(r"^(.*)_\d{4}-\d{2}-\d{2}-\d{4}-\d{2}-\d{2}$", zarr_path.stem)
base_name = match.group(1) if match else zarr_path.stem

# Select only timesteps between March 1st and May 31st, 2024
ds = ds.sel(time=slice("2024-03-01", "2024-05-31"))

# Inspect the dataset
ds

## 🗂️ **4. Loop Through Timesteps and Export Rasters**

In [ ]:
# Loop through timesteps
for i, timestamp in enumerate(ds["time"].values):
    try:
        da = ds["chl"].sel(time=timestamp)
        date_str = pd.to_datetime(str(timestamp)).strftime("%Y-%m-%d")

        da.rio.write_crs("EPSG:4326", inplace=True)
        output_path = output_dir / f"{base_name}_{date_str}.tif"
        da.rio.to_raster(output_path)

        logger.success(f"🎉 Saved {output_path.name}")

    except Exception as e:
        logger.error(f"🚩 Failed to export timestep {timestamp}: {e}")


## 🧭 **5. Next Steps**

You’ve now exported each daily timestep of chlorophyll-a data as a **GeoTIFF**.

In the next step, we’ll bring these rasters into **QGIS**, where you’ll:

- Apply **styling** (e.g., color ramps) to visualize bloom dynamics
- Use the **Print Layout** to export frames or create an animation
- Automate styling using the QGIS Python Console/Python script provided in the repository

📂 The QGIS automation script lives here in your project:  
`qgis-to-illustrator-tools/qgis/qgis_layout_export.py`

Let's go!
